In [51]:
import pandas as pd
import numpy as np
import seaborn as sb
import tensorflow as tf
import sklearn
import joblib
import pickle

In [3]:
anime = pd.read_csv("Anime.csv")

In [4]:
anime.head(2)

,ID,Title,Synonyms,Japanese,English,Synopsis,Type,Episodes,Status,Start_Aired,...,Themes,Demographics,Duration_Minutes,Rating,Score,Scored_Users,Ranked,Popularity,Members,Favorites
0,16498,Shingeki no Kyojin,"AoT, SnK",進撃の巨人,Attack on Titan,"Centuries ago, mankind was slaughtered to near...",TV,25.0,Finished Airing,"Apr 7, 2013",...,"Gore, Military, Survival",Shounen,24.0,R - 17+ (violence & profanity),8.531,519803.0,1002.0,1,3524109,155695
1,1535,Death Note,DN,デスノート,Death Note,"Brutal murders, petty thefts, and senseless vi...",TV,37.0,Finished Airing,"Oct 4, 2006",...,Psychological,Shounen,23.0,R - 17+ (violence & profanity),8.621,485487.0,732.0,2,3504535,159701


In [5]:
anime["Studios"]

0          Wit Studio
1            Madhouse
2               Bones
3            Madhouse
4        A-1 Pictures
             ...     
21455         Unknown
21456         Unknown
21457         Unknown
21458         Unknown
21459         Unknown
Name: Studios, Length: 21460, dtype: object

In [6]:
anime.columns

Index(['ID', 'Title', 'Synonyms', 'Japanese', 'English', 'Synopsis', 'Type',
       'Episodes', 'Status', 'Start_Aired', 'End_Aired', 'Premiered',
       'Broadcast', 'Producers', 'Licensors', 'Studios', 'Source', 'Genres',
       'Themes', 'Demographics', 'Duration_Minutes', 'Rating', 'Score',
       'Scored_Users', 'Ranked', 'Popularity', 'Members', 'Favorites'],
      dtype='object')

In [7]:
#Selcting the columns on which our recommendation system will work on
#ID
#Title 
#English
#Studios

#Synopsis
#Demographics
#Genres
#Themes

anime = anime[["ID","Title","English","Synopsis","Studios","Genres","Themes","Demographics"]]

In [8]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21460 entries, 0 to 21459
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            21460 non-null  int64 
 1   Title         21460 non-null  object
 2   English       21460 non-null  object
 3   Synopsis      21460 non-null  object
 4   Studios       21460 non-null  object
 5   Genres        21460 non-null  object
 6   Themes        21460 non-null  object
 7   Demographics  21460 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.3+ MB


In [9]:
anime.isna().sum()

ID              0
Title           0
English         0
Synopsis        0
Studios         0
Genres          0
Themes          0
Demographics    0
dtype: int64

In [10]:
anime.head()

,ID,Title,English,Synopsis,Studios,Genres,Themes,Demographics
0,16498,Shingeki no Kyojin,Attack on Titan,"Centuries ago, mankind was slaughtered to near...",Wit Studio,"Action, Drama","Gore, Military, Survival",Shounen
1,1535,Death Note,Death Note,"Brutal murders, petty thefts, and senseless vi...",Madhouse,"Supernatural, Suspense",Psychological,Shounen
2,5114,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist Brotherhood,After a horrific alchemy experiment goes wrong...,Bones,"Action, Adventure, Drama, Fantasy",Military,Shounen
3,30276,One Punch Man,One Punch Man,The seemingly unimpressive Saitama has a rathe...,Madhouse,"Action, Comedy","Parody, Super Power",Seinen
4,11757,Sword Art Online,Sword Art Online,Ever since the release of the innovative Nerve...,A-1 Pictures,"Action, Adventure, Fantasy, Romance","Love Polygon, Video Game",Unknown


In [11]:
anime["Synopsis"] = anime["Synopsis"].apply(lambda x: x.split())

In [12]:
anime.head()

,ID,Title,English,Synopsis,Studios,Genres,Themes,Demographics
0,16498,Shingeki no Kyojin,Attack on Titan,"[Centuries, ago,, mankind, was, slaughtered, t...",Wit Studio,"Action, Drama","Gore, Military, Survival",Shounen
1,1535,Death Note,Death Note,"[Brutal, murders,, petty, thefts,, and, sensel...",Madhouse,"Supernatural, Suspense",Psychological,Shounen
2,5114,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist Brotherhood,"[After, a, horrific, alchemy, experiment, goes...",Bones,"Action, Adventure, Drama, Fantasy",Military,Shounen
3,30276,One Punch Man,One Punch Man,"[The, seemingly, unimpressive, Saitama, has, a...",Madhouse,"Action, Comedy","Parody, Super Power",Seinen
4,11757,Sword Art Online,Sword Art Online,"[Ever, since, the, release, of, the, innovativ...",A-1 Pictures,"Action, Adventure, Fantasy, Romance","Love Polygon, Video Game",Unknown


In [13]:
anime["Studios"] = anime["Studios"].str.replace(" ","")

In [14]:
def Convert(string):
    li = list(string.split(" "))
    return li

In [15]:
anime["Studios"] = anime["Studios"].apply(Convert)
anime["Demographics"] = anime["Demographics"].apply(Convert)

In [16]:
anime["Studios"][0]
anime["Demographics"][0]

['Shounen']

In [17]:

anime["Genres"] = anime["Genres"].apply(lambda x : x.split())
anime["Themes"] = anime["Themes"].apply(lambda x : x.split())

In [18]:
anime["Studios"]

0          [WitStudio]
1           [Madhouse]
2              [Bones]
3           [Madhouse]
4        [A-1Pictures]
             ...      
21455        [Unknown]
21456        [Unknown]
21457        [Unknown]
21458        [Unknown]
21459        [Unknown]
Name: Studios, Length: 21460, dtype: object

In [19]:
anime.head()

,ID,Title,English,Synopsis,Studios,Genres,Themes,Demographics
0,16498,Shingeki no Kyojin,Attack on Titan,"[Centuries, ago,, mankind, was, slaughtered, t...",[WitStudio],"[Action,, Drama]","[Gore,, Military,, Survival]",[Shounen]
1,1535,Death Note,Death Note,"[Brutal, murders,, petty, thefts,, and, sensel...",[Madhouse],"[Supernatural,, Suspense]",[Psychological],[Shounen]
2,5114,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist Brotherhood,"[After, a, horrific, alchemy, experiment, goes...",[Bones],"[Action,, Adventure,, Drama,, Fantasy]",[Military],[Shounen]
3,30276,One Punch Man,One Punch Man,"[The, seemingly, unimpressive, Saitama, has, a...",[Madhouse],"[Action,, Comedy]","[Parody,, Super, Power]",[Seinen]
4,11757,Sword Art Online,Sword Art Online,"[Ever, since, the, release, of, the, innovativ...",[A-1Pictures],"[Action,, Adventure,, Fantasy,, Romance]","[Love, Polygon,, Video, Game]",[Unknown]


In [20]:
anime["tag"] = anime["Synopsis"]+anime["Studios"]+anime["Genres"]+anime["Themes"]+anime["Demographics"]

In [21]:
anime.head()

,ID,Title,English,Synopsis,Studios,Genres,Themes,Demographics,tag
0,16498,Shingeki no Kyojin,Attack on Titan,"[Centuries, ago,, mankind, was, slaughtered, t...",[WitStudio],"[Action,, Drama]","[Gore,, Military,, Survival]",[Shounen],"[Centuries, ago,, mankind, was, slaughtered, t..."
1,1535,Death Note,Death Note,"[Brutal, murders,, petty, thefts,, and, sensel...",[Madhouse],"[Supernatural,, Suspense]",[Psychological],[Shounen],"[Brutal, murders,, petty, thefts,, and, sensel..."
2,5114,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist Brotherhood,"[After, a, horrific, alchemy, experiment, goes...",[Bones],"[Action,, Adventure,, Drama,, Fantasy]",[Military],[Shounen],"[After, a, horrific, alchemy, experiment, goes..."
3,30276,One Punch Man,One Punch Man,"[The, seemingly, unimpressive, Saitama, has, a...",[Madhouse],"[Action,, Comedy]","[Parody,, Super, Power]",[Seinen],"[The, seemingly, unimpressive, Saitama, has, a..."
4,11757,Sword Art Online,Sword Art Online,"[Ever, since, the, release, of, the, innovativ...",[A-1Pictures],"[Action,, Adventure,, Fantasy,, Romance]","[Love, Polygon,, Video, Game]",[Unknown],"[Ever, since, the, release, of, the, innovativ..."


In [22]:
anime = anime[["ID","Title","English","tag"]]

In [23]:
anime.head()

,ID,Title,English,tag
0,16498,Shingeki no Kyojin,Attack on Titan,"[Centuries, ago,, mankind, was, slaughtered, t..."
1,1535,Death Note,Death Note,"[Brutal, murders,, petty, thefts,, and, sensel..."
2,5114,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist Brotherhood,"[After, a, horrific, alchemy, experiment, goes..."
3,30276,One Punch Man,One Punch Man,"[The, seemingly, unimpressive, Saitama, has, a..."
4,11757,Sword Art Online,Sword Art Online,"[Ever, since, the, release, of, the, innovativ..."


In [24]:
anime["tag"] = anime["tag"].apply(lambda x: " ".join(x))

In [25]:
anime["tag"] = anime["tag"].apply(lambda x: x.lower())

In [26]:
anime.tag[0]

'centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. what makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out of pleasure. to ensure their survival, the remnants of humanity began living within defensive barriers, resulting in one hundred years without a single titan encounter. however, that fragile calm is soon shattered when a colossal titan manages to breach the supposedly impregnable outer wall, reigniting the fight for survival against the man-eating abominations. after witnessing a horrific personal loss at the hands of the invading creatures, eren yeager dedicates his life to their eradication by enlisting into the survey corps, an elite military unit that combats the merciless humanoids outside the protection of the walls. eren, his adopted sister mikasa ackerman, and his childhood friend a

In [27]:
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()

In [28]:
def lema(keyword):
    l = []
    for i in keyword.split():
        l.append(lm.lemmatize(i))
    return " ".join(l) 

In [29]:
anime["tag"][0]

'centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. what makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out of pleasure. to ensure their survival, the remnants of humanity began living within defensive barriers, resulting in one hundred years without a single titan encounter. however, that fragile calm is soon shattered when a colossal titan manages to breach the supposedly impregnable outer wall, reigniting the fight for survival against the man-eating abominations. after witnessing a horrific personal loss at the hands of the invading creatures, eren yeager dedicates his life to their eradication by enlisting into the survey corps, an elite military unit that combats the merciless humanoids outside the protection of the walls. eren, his adopted sister mikasa ackerman, and his childhood friend a

In [30]:
anime["tag"]  = anime["tag"].apply(lema)

In [31]:
anime.head()

,ID,Title,English,tag
0,16498,Shingeki no Kyojin,Attack on Titan,"century ago, mankind wa slaughtered to near ex..."
1,1535,Death Note,Death Note,"brutal murders, petty thefts, and senseless vi..."
2,5114,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist Brotherhood,after a horrific alchemy experiment go wrong i...
3,30276,One Punch Man,One Punch Man,the seemingly unimpressive saitama ha a rather...
4,11757,Sword Art Online,Sword Art Online,ever since the release of the innovative nerve...


# Turning Text into vectors and deleting stop words from the tag column

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000,stop_words = "english")

In [34]:
doc_vec = cv.fit_transform(anime["tag"]).toarray()

In [35]:
doc_vec.shape

(21460, 5000)

In [37]:
cv.get_feature_names()

C:\Users\Dell\anaconda3\envs\mypython\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000',
 '10',
 '100',
 '10th',
 '11',
 '11th',
 '12',
 '12th',
 '13',
 '13th',
 '14',
 '14th',
 '15',
 '15th',
 '16',
 '16th',
 '17',
 '18',
 '19',
 '1999',
 '19th',
 '1pictures',
 '1st',
 '20',
 '2000',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '20th',
 '21',
 '21st',
 '22',
 '22nd',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '2d',
 '2nd',
 '30',
 '300',
 '3d',
 '3rd',
 '4th',
 '50',
 '500',
 '5th',
 '6th',
 '7th',
 '8bit',
 '8th',
 'abandon',
 'abandoned',
 'abducted',
 'abilities',
 'ability',
 'able',
 'abnormal',
 'aboard',
 'abroad',
 'abruptly',
 'absence',
 'absent',
 'absolute',
 'absolutely',
 'abstract',
 'absurd',
 'abuse',
 'abyss',
 'academic',
 'academy',
 'accept',
 'accepted',
 'accepting',
 'accepts',
 'access',
 'accident',
 'accidentally',
 'accompanied',
 'accompany',
 'accompanying',
 'accomplish',
 'according',
 'account',
 'accused',
 'accust

In [38]:
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
similarity = cosine_similarity(doc_vec)

In [41]:
similarity

array([[1.        , 0.11514155, 0.1524679 , ..., 0.        , 0.        ,
        0.        ],
       [0.11514155, 1.        , 0.13218176, ..., 0.        , 0.        ,
        0.        ],
       [0.1524679 , 0.13218176, 1.        , ..., 0.04355036, 0.04474374,
        0.0358748 ],
       ...,
       [0.        , 0.        , 0.04355036, ..., 1.        , 0.97332853,
        0.78039897],
       [0.        , 0.        , 0.04474374, ..., 0.97332853, 1.        ,
        0.80178373],
       [0.        , 0.        , 0.0358748 , ..., 0.78039897, 0.80178373,
        1.        ]])

In [99]:
similarity.shape

(21460, 21460)

In [42]:
anime.shape

(21460, 4)

In [113]:
anime[anime['English']==""]

,ID,Title,English,tag


In [44]:
anime.head()

,ID,Title,English,tag
0,16498,Shingeki no Kyojin,Attack on Titan,"century ago, mankind wa slaughtered to near ex..."
1,1535,Death Note,Death Note,"brutal murders, petty thefts, and senseless vi..."
2,5114,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist Brotherhood,after a horrific alchemy experiment go wrong i...
3,30276,One Punch Man,One Punch Man,the seemingly unimpressive saitama ha a rather...
4,11757,Sword Art Online,Sword Art Online,ever since the release of the innovative nerve...


In [100]:
def recommend(movie):
    index = anime[anime["English"] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:10]:
        print(anime.iloc[i[0]].English)

In [101]:
recommend("Dragon Ball Z")

Dragon Ball Z Battle of Gods
Dragon Ball Z Kai
Dragon Ball Z Resurrection 'F'
Dragon Ball GT
Dragon Ball Z Cooler's Revenge
Dragon Ball Z Fusion Reborn
Dragon Ball Super
Dragon Ball Super Broly
Dragon Ball


In [86]:
recommend("Naruto")

Boruto Naruto Next Generations
Naruto Shippuden
Boruto Naruto the Movie
Naruto Shippuden Chunin Exam on Fire! and Naruto vs. Konohamaru!
Naruto The Lost Story - Mission Protect the Waterfall Village
Naruto Shippuden the Movie 2 -Bonds-
Naruto Shippuden the Movie The Lost Tower
Naruto the Movie Blood Prison
Naruto Shippuden Konoha Gakuen - Special


In [58]:
recommend("One Piece")

One Piece: Episode of East Blue - Luffy to 4-nin no Nakama no Daibouken
One Piece Movie 14: Stampede
One Piece Film: Z
One Piece Movie 01
One Piece: Romance Dawn Story
One Piece Film: Strong World
One Piece: Oounabara ni Hirake! Dekkai Dekkai Chichi no Yume!
One Piece Film: Strong World Episode 0
One Piece Movie 06: Omatsuri Danshaku to Himitsu no Shima


In [59]:
recommend("Death Note")

Death Note: Rewrite
Ghost Messenger
Wan Jie Shen Zhu
Sekaikei Sekai Ron
Da Yu Hai Tang (Movie)
Yadamon
Blood Lad
Dr. Stone
Matantei Loki Ragnarok


In [60]:
recommend("Berserk")

Berserk: Ougon Jidai-hen I - Haou no Tamago
Berserk: Ougon Jidai-hen III - Kourin
Hibike! Euphonium 2
Hibike! Euphonium Movie 2: Todoketai Melody
Wake Up, Girls!
Hibike! Euphonium Movie 3: Chikai no Finale
Hibike! Euphonium Movie 1: Kitauji Koukou Suisougaku-bu e Youkoso
Hibike! Euphonium: Kakedasu Monaka
BanG Dream! Movie: Episode of Roselia - I: Yakusoku


In [102]:
recommend("Attack on Titan")

Attack on Titan Season 2
Attack on Titan The Roar of Awakening
Attack on Titan Season 3
Attack on Titan The Final Season Part 2
Attack on Titan The Final Season Part 3
Attack on Titan Season 3 Part 2
Attack on Titan OAD
Attack on Titan Chronicle
Attack on Titan Crimson Bow and Arrow


In [116]:
recommend("One Punch Man")

Unknown
One Punch Man Season 2
Tiger & Bunny
My Hero Academia
Samurai Flamenco
Ooya-san wa Shishunki!
Yusibu I couldn't become a hero, so I reluctantly decided to get a job.
Unknown
Z Rangers


In [117]:
recommend("My Hero Academia")

Tiger & Bunny
My Hero Academia Season 6
My Hero Academia Two Heroes Specials
My Hero Academia Season 4
Campus Special Investigator Hikaruon
My Hero Academia Season 2 Hero Notebook
Yuki Yuna is a Hero
My Hero Academia Rescue! Rescue Training
Unknown
